In [ ]:
import numpy as np 
import scipy.fft as fft
from pyevtk.hl import imageToVTK

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
ndim        = 3
grid_levels = np.array([7,7,5]) 
grid_shape  = 2**grid_levels
domain_size = np.array([1, 1, 0.25 ]) 

friction_velocity = 2.683479938442173
reference_height  = 180.0
roughness_height  = 0.75
prefactor         = 3.2 * friction_velocity**2 * reference_height**(-2/3)
Lengthscale       = 0.59 * reference_height

In [ ]:
def VonKarmanSpectralTensor(prefactor=1, beta=None):

    Nd, d          = grid_shape.copy(), ndim
    frequences     = [fft.fftfreq(Nd[j], domain_size[j]/Nd[j]) for j in range(d)]
    Nd[-1]         = int(Nd[-1] // 2)+1 
    frequences[-1] = frequences[-1][:Nd[-1]]

    k  = np.array(list(np.meshgrid(*frequences, indexing='ij')))
    kk = np.sum(k**2, axis=0)

    ### curl
    SqrtSpectralTens = np.tile(np.zeros(Nd, dtype=np.complex128),(3,3,1,1,1)) 
    SqrtSpectralTens[0,1,...] = -k[2,...]
    SqrtSpectralTens[0,2,...] =  k[1,...]
    SqrtSpectralTens[1,0,...] =  k[2,...]
    SqrtSpectralTens[1,2,...] = -k[0,...]
    SqrtSpectralTens[2,0,...] = -k[1,...]
    SqrtSpectralTens[2,1,...] =  k[0,...]
    SqrtSpectralTens *= 1j

    ### energy spectrum
    L                  = Lengthscale
    const              = prefactor * (L**(17/3)) / (4*np.pi)
    SqrtEnergySpectrum = np.sqrt( const / (1 + (L**2) * kk)**(17/6) )
    SqrtSpectralTens  *= SqrtEnergySpectrum

    ### dissipation (due to diffusion)
    if beta: SqrtSpectralTens *= np.exp(-beta*np.sqrt(kk))

    return SqrtSpectralTens

In [ ]:
noise           = np.random.normal(loc=0, scale=1, size=[ndim]+list(grid_shape))
noise_hat       = fft.rfftn(noise, norm="ortho")
kernel_hat      = VonKarmanSpectralTensor(prefactor=prefactor)
fluctuation_hat = (kernel_hat * noise_hat).sum(axis=1) ### matrix-vector product
fluctuation     = fft.irfftn(fluctuation_hat, norm="ortho")
spacing         = list(domain_size/grid_shape) #[1/grid_shape.max()]*ndim

## Write out Fluctuation Data to VTK

In [ ]:
filename = "./results/fluctuation_simple"
cellData = {'fluctuation field' : tuple(fluctuation)}

imageToVTK(filename, cellData=cellData, spacing=spacing)

## Plot Fluctuations in Plotly

In [ ]:
x = np.array([spacing[0]*n for n in range(fluctuation.shape[1])])
y = np.array([spacing[1]*n for n in range(fluctuation.shape[2])])
z = np.array([spacing[2]*n for n in range(fluctuation.shape[3])])

X, Y, Z = np.meshgrid(x, y, z)

fig = make_subplots(
    rows = 1, cols = 3, 
    subplot_titles=("x Component", "y Component", "z Component"), 
    specs= [[{'type': 'volume'}, {'type': 'volume'}, {'type': 'volume'}]]
)

fig.add_trace(go.Volume(
        x = X.flatten(), 
        y = Y.flatten(), 
        z = Z.flatten(), 
        value = fluctuation[0].flatten(),
        coloraxis='coloraxis',
        opacity=0.5
    ), row = 1, col = 1
)

fig.add_trace(go.Volume(
        x = X.flatten(), 
        y = Y.flatten(), 
        z = Z.flatten(), 
        value = fluctuation[1].flatten(),
        coloraxis='coloraxis',
        opacity=0.5
    ), row = 1, col = 2
)

fig.add_trace(go.Volume(
        x = X.flatten(), 
        y = Y.flatten(), 
        z = Z.flatten(), 
        value = fluctuation[2].flatten(),
        coloraxis='coloraxis',
        opacity=0.5
    ), row = 1, col = 3
)

fig.update_layout(scene = dict( aspectmode='data'), scene2 = dict( aspectmode='data'), scene3 = dict( aspectmode='data')) 

fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        ), 
    scene2 = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        ), 
    scene3 = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        )
    )

fig.update_coloraxes(colorscale='spectral')

fig.show()

In [ ]:
fluctuation_magnitude = np.sqrt(fluctuation[0]**2+fluctuation[1]**2+fluctuation[2]**2)

fig = go.Figure(
    data = go.Volume(
        x=X.flatten(), 
        y=Y.flatten(), 
        z=Z.flatten(), 
        value=fluctuation_magnitude.flatten(), 
        opacity=0.5, 
        colorscale='spectral', 
        colorbar={"title": '|U(x)|'}
    ), 
)

fig.update_layout(scene = dict( aspectmode='data')) 

fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis = dict(visible=False)
        ), 
)

fig.update_layout(title_text="Fluctuation Vector Magnitude", title_x=0.5)

fig.show()